In [3]:
import os
import json
import cv2
import numpy as np
from tensorflow.keras.models import load_model

MODEL_PATH = "face_recognition_model.h5"
CLASS_INDICES_PATH = "class_indices.json"

def load_class_indices():
    if not os.path.exists(CLASS_INDICES_PATH):
        print("Class indices not found. Train the model first.")
        return None
    with open(CLASS_INDICES_PATH, 'r') as f:
        return json.load(f)

def capture_image():
    input("Press Enter to start capturing image...")
    cap = cv2.VideoCapture(0)
    print("Press 'q' to capture image.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break
        
        cv2.imshow("Authenticate", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            return frame

def preprocess_image(image):
    image = cv2.resize(image, (128, 128))
    image = image.astype('float32') / 255.0
    return np.expand_dims(image, axis=0)

def authenticate_user():
    if not os.path.exists(MODEL_PATH):
        print("Model not found. Train the model first.")
        return
    
    class_indices = load_class_indices()
    if class_indices is None:
        return
    
    model = load_model(MODEL_PATH)
    frame = capture_image()
    if frame is None:
        print("No image captured.")
        return
    
    image = preprocess_image(frame)
    predictions = model.predict(image)
    user_index = np.argmax(predictions)
    confidence = np.max(predictions)
    
    # Map numeric index back to user name
    user_map = {v: k for k, v in class_indices.items()}
    user_name = user_map.get(user_index, "Unknown")
    
    if confidence > 0.8:
        print(f"Authenticated: {user_name} (Confidence: {confidence:.2f})")
        print("Gate Opening...")
    else:
        print("Authentication Failed. Access Denied.")

if __name__ == "__main__":
    print("Face Authentication System")
    input("Press Enter to start authentication...")
    authenticate_user()


Face Authentication System


Press Enter to start authentication... 


Press Enter to start capturing image... 


Press 'q' to capture image.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
Authenticated: mohini (Confidence: 1.00)
Gate Opening...
